In [108]:
from gspread_dataframe import *
from gspread_formatting import *
import imp
import pandas as pd
import pickle as pkl
import re
import numpy as np

In [109]:
d = imp.load_source('dashboard', 'dashboard.py')
g = imp.load_source('gsync', 'modules/gCanvas.py')
c = imp.load_source('conf', 'config/conf.py')
r = imp.load_source('report', 'modules/reader.py')
s = imp.load_source('patch', 'modules/patcher.py')
t = imp.load_source('triggers', 'config/triggers.py')

In [110]:
conn = g.Connection("new test dashboard")
name = "3. Триггеры — стат. нед."
ws = conn.get_sheet_by_name(name)
gdf = get_as_dataframe(ws, index_col = 0)

In [111]:
df = r.get_report('all.xlsx', 0)
df = df[df.channel != 'Ручные рассылки']

In [112]:
patcher = s.Patcher(df, 2, 24, gdf)

Определите категорию кампании для 9 новых:

Нажмите 'Enter' для Email
	'1' для Web-push
	'2' для сезонной
	'3' для SMS

-> Триггер 17. Брошенная корзина с оффером: 

Нажмите только 'Enter', если название компании Вас устраивает:
Введите новое название для отображения в Google Sheets
Брошенная корзина с оффером
Нажмите 'Enter' для Email
	'1' для Web-push
	'2' для сезонной
	'3' для SMS

-> NY landing 2020 DOI: 
2
Нажмите только 'Enter', если название компании Вас устраивает:
Введите новое название для отображения в Google Sheets

Нажмите 'Enter' для Email
	'1' для Web-push
	'2' для сезонной
	'3' для SMS

-> NY landing 2020 DOI promocode: 
2
Нажмите только 'Enter', если название компании Вас устраивает:
Введите новое название для отображения в Google Sheets

Нажмите 'Enter' для Email
	'1' для Web-push
	'2' для сезонной
	'3' для SMS

-> Триггер_14_ "Онлайн дешевле": 

Нажмите только 'Enter', если название компании Вас устраивает:
Введите новое название для отображения в Google Sheets
Онлай

In [95]:
out = s.get_dictionary('config/data/outlay3.pkl')

In [115]:
import numpy as np

In [117]:
z = s.get_dict_value('/triggers.pkl', 2)

In [ ]:
def slice_df(df):
    outlay = get_dict('config/data/outlay3.pkl')
    main = gdf.iloc[:outlay['email']]
    email = gdf.iloc[outlay['email']:outlay['wp']]
    wp = gdf.iloc[outlay['wp']:outlay['seasonal']]
    seasonal = gdf.iloc[outlay['seasonal']:outlay['sms']]
    

In [123]:
outlay = {}
tmp = patcher.gdf.iloc[:, 0]
for channel in z.keys():
    t = list(z[channel].values())[0]
    outlay[channel] = np.where(tmp == t)[0][0]

In [128]:
email = patcher.gdf.iloc[outlay['email']:outlay['wp']]

In [163]:
indices = email.iloc[:, 0].loc[email.iloc[:, 0].where(email.iloc[:, 0] == 'Доставлено').notna()].index.values

In [156]:
sum(patcher.gdf.iloc[email.iloc[:, 0].loc[email.iloc[:, 0].where(email.iloc[:, 0] == 'Доставлено').notna()].index.values, 23])

25130.0

In [178]:
patcher.gdf.iloc[:,[0,23]].to_csv('tmp.csv', index= False)

In [186]:
get_attr_sum(patcher.gdf, 'email', 23, 'Доставлено')

25130.0

In [187]:
get_attr_sum(patcher.gdf, 'email', 23, 'Открытия')

9496.0

In [198]:
def get_attr_sum(gdf, channel, max_cols, attr, outlay):
    if channel == 'email':
        df = gdf.iloc[outlay['email']:outlay['wp']]
    elif channel == 'wp':  
        df = gdf.iloc[outlay['wp']:outlay['seasonal']]
    elif channel == 'seasonal':
        df = gdf.iloc[outlay['seasonal']:outlay['sms']]
    indices = df.iloc[:, 0].loc[df.iloc[:, 0].where(df.iloc[:, 0] == attr).notna()].index.values
    return sum(gdf.iloc[indices, max_cols])

class mainStats():
    def __init__(self, gdf, channel, traffic):
        self.delivered = get_attr_sum(gdf, channel, maxrows, 'Доставлено')
        self.opened = get_attr_sum(gdf, channel, maxrows, 'Открытия')
        self.clicked = get_attr_sum(gdf, channel, maxrows, 'Кликов')
        self.revenue = get_attr_sum(gdf, channel, maxrows, 'Доход по атрибуции MB')
        self.orders = get_attr_sum(gdf, channel, maxrows, 'Покупок по атрибуции MB')
        self.openr = self.opened/self.delivered
        self.clickr = self.clicked/self.opened
        self.ord_traffic = self.orders/traffic
        self.ord_conversion = self.orders/self.delivered

In [231]:
patcher.gdf.iloc[:, 0]

0                  номер недели
1                          даты
2               Трафик на сайте
3                   Регистраций
4                         Email
                 ...           
577       Доход по атрибуции MB
578     Покупок по атрибуции MB
579    Средний чек по атрибуции
580          Конверсия в заказы
581                         NaN
Name:  , Length: 582, dtype: object

In [226]:
get_value_index(patcher.gdf.iloc[:outlay['email'], 0], 'Email')

4

In [ ]:
def fill_main(gdf, indices, max_cols):
    

In [ ]:
def update_campaigns(gdf, max_cols):
    """
    -> gdf: dataframe of main table
    """
    channels = ['Email', 'Web-push', 'Сезонные триггеры']
    main_=['Доставлено', 'OR', 'CR', 'Выручка', 'Заказов',
           'Заказов от трафика', 'Конверсия в заказы',]
    
    def get_value_index(series, value):
        series = series.where(series == value)
        return series.loc[series.notnull()].index[0]
    
    indices = []
    for channel in channels:
        indices.append(get_value_index(gdf.iloc[:, 0], max_cols))
    
    

* global traffic
* get main через named tuple
* написать метод get_as_dataframe для gCanvas
    * *перед удалением всех 4 групп, нужно реализовать суммы по группам*
    * *рекурсивный обход df с печатью формата*


In [ ]:
main_={'Доставлено':'Доставлено',
       'OR':'OR',
       'CR':'CR',
       'Доход по атрибуции MB':'Выручка',
       'Заказов':'Покупок по атрибуции MB',
       'Заказов':'Заказов от трафика',
       'Конверсия в заказы':'Конверсия в заказы',}

In [100]:
from collections import namedtuple

In [106]:
Point = namedtuple('group', [
        'Доставлено',
        'OR',
        'CR',
        'Выручка',
        'Заказов',
        'Заказов от трафика',
        'Конверсия в заказы'])

ValueError: Type names and field names must be valid identifiers: 'Заказов от трафика'

In [69]:
def get_index(chnl):
    if chnl == 'email':
        return 6
    elif chnl == 'wp':
        return 16
    elif chnl == 'seasonal':
        return 24
    else : return None
        

In [ ]:
main_=[] 
for channel in t.triggers.keys():
        trigger = list(t.triggers[channel].values())[0]
        outlay[channel] = df[df.columns[0]][df[df.columns[0]] == trigger].index[0]

In [ ]:
s.put_dictionary('config/dictionaries/outlay3.pkl', outlay)

In [ ]:
main_={'Доставлено':'Доставлено',
       'OR':'OR',
      'CR':'CR',
      'Доход по атрибуции MB':'Выручка',
      'Заказов':'Покупок по атрибуции MB',
      'Заказов':'Заказов от трафика',
      'Конверсия в заказы':'Конверсия в заказы',}

In [275]:
def main_tbl_aggregator(df, attr, max_cols):
    n = 0
    res = 0
    if attr in ['Доставлено',  'Доход по атрибуции MB', 'Покупок по атрибуции MB',]:
        res+=df.loc[df.name == attr, max_cols]
    elif attr in ['OR','CR', 'Конверсия в заказы']:
        
    

In [185]:
z = patcher.gdf.reset_index()

In [276]:
def get_campaign_info(df, name, column):
    """Locates specific value for named campaign"""
    try: res = df.loc[df.name == name, column].values[0]
    except: res = 0
    return res

In [214]:
df = r.get_report('data/report/all.xlsx', 0)
df = df[df.channel != 'Ручные рассылки']

In [215]:
df.columns = df.columns.to_series().replace(online_attr)

In [230]:
df.columns

Index(['name', 'channel', 'subject', 'preview_url', 'sent', 'Доставлено',
       'deliver_rate', 'Открытия', 'OR', 'Кликов', 'CR', 'CTR', 'Отписок',
       'UR', 'Доход по атрибуции MB', 'Покупок по атрибуции MB',
       'Средний чек по атрибуции', 'Конверсия в заказы'],
      dtype='object')

In [247]:
online_attr

{'delivered': 'Доставлено',
 'opened': 'Открытия',
 'open_rate': 'OR',
 'clicked': 'Кликов',
 'click_rate': 'CR',
 'CTR': 'CTR',
 'unfollowed': 'Отписок',
 'unfollow_rate': 'UR',
 'revenue': 'Доход по атрибуции MB',
 'orders': 'Покупок по атрибуции MB',
 'avg_bill': 'Средний чек по атрибуции',
 'order_conversion': 'Конверсия в заказы'}

In [216]:
df.loc[df.name == 'Для вас сюрприз! Откройте письмо 😉', 'Доставлено']

104    2191
Name: Доставлено, dtype: int64

In [279]:
z.iloc[index, 23]

nan

In [271]:
online_attr

{'delivered': 'Доставлено',
 'opened': 'Открытия',
 'open_rate': 'OR',
 'clicked': 'Кликов',
 'click_rate': 'CR',
 'CTR': 'CTR',
 'unfollowed': 'Отписок',
 'unfollow_rate': 'UR',
 'revenue': 'Доход по атрибуции MB',
 'orders': 'Покупок по атрибуции MB',
 'avg_bill': 'Средний чек по атрибуции',
 'order_conversion': 'Конверсия в заказы'}

In [280]:
def get_campaign_info(df, name, column):
    """Locates specific value for named campaign"""
    try: res = df.loc[df.name == name, column].values[0]
    except: return 0
    return res
for channel in t.triggers.keys() :
    for trigger, campaign in t.triggers[channel].items():
        val = z[z.columns[0]].loc[z[z.columns[0]] == campaign].index[0]
        for (index, row), (column) in zip(z.iloc[val+1:].iterrows(), online_attr.values()):
            if pd.isnull(row[0]):
                break
            elif column == row[0]:
                val = df.loc[df.name == trigger, column].values[0]
                if 
                z.iloc[index,23] = df.loc[df.name == trigger, column].values[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
z

In [232]:
z.iloc[379,22]

nan

In [ ]:
z

In [59]:
df = get_report('data/report/all.xlsx', 'Свод. данные (online + offline)')

In [3]:
%%time
!python dashboard.py -p 1 -r "data/report.xlsx" -o 'data/orders.csv' -c 'data/client_groups.f'

dashboard.py:3: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
CPU times: user 112 ms, sys: 12 ms, total: 124 ms
Wall time: 5.96 s
